In [1]:
import sys
import os
import numpy as np                                # numpy is a collection of python modules for array math.
from matplotlib import pyplot as plt              # A collection of modules. Needed here to display images and graphs.
from matplotlib import pylab                      # A collection of modules. Needed here to plot histograms.
sys.path.append('/Users/benkoch/Desktop/UChicago/2nd year/Spring/ASTR 21200/a212pylibs') # Adds the path to the DataPype module to the system path.
from datapype.datafits import DataFits            # Gets the function that makes datapype io objects from module datafits in package datapype.
from astropy.io import fits                       # Need this if you want to use astropy.io io objects.
from ipywidgets import interact                   # Need this for interactive plots.
from matplotlib.colors import LogNorm             # Machinery for LogNorm scaling of intensities.
from matplotlib.colors import SymLogNorm          # Machinery for SymLogNorm scaling of intensities.
from matplotlib.colors import PowerNorm           # Machinery for LogNorm (e.g., square root) scaling of intensities.
from astropy.stats import mad_std                 # The median absolute deviation, a more robust estimator than std.
import scipy.ndimage as nd                        # Various algorithms for image transformations.


# ## Use this if dah_functions is a subdirectory of a212pylibs.
# from dah_functions.dah_functions import head
# from dah_functions.dah_functions import displaypic
# from dah_functions.dah_functions import displaypic2
# from dah_functions.dah_functions import show_image
# from dah_functions.dah_functions import blink_images
# from dah_functions.dah_functions import blink2images
# from dah_functions.dah_functions import rowplot
# from dah_functions.dah_functions import colplot
# from dah_functions.dah_functions import rowcolplot

## Use this if dah_functions is stored in a different path from a212pylibs.
## The following adds a bunch of functions in a module stored in the path below.
sys.path.append('/Users/benkoch/Desktop/UChicago/2nd year/Spring/ASTR 21200/a212pylibs/dah_functions') # Adds path to MyCode directory on my computer to system python path.
from dah_functions_2 import head
from dah_functions_2 import displaypic
from dah_functions_2 import displaypic2
from dah_functions_2 import show_image
from dah_functions_2 import blink_images
from dah_functions_2 import rowplot
from dah_functions_2 import colplot
from dah_functions_2 import rowcolplot
from dah_functions_2 import moments
from dah_functions_2 import quickpic
from dah_functions_2 import getpatch
from dah_functions_2 import stackfits

###############################################################################
# ## Don't need the following in this notebook. Hold on to them for now.
# import time        
# import re
# import scipy.stats
# import matplotlib.colors as colors
# import configobj  #What is this? Where does it come from?
###############################################################################

In [2]:
len(get_ipython().user_ns.keys())

48

In [42]:
datapath = '/Users/benkoch/Desktop/UChicago/2nd year/Spring/ASTR 21200/24inch_data/190526/dark'
median_path = '/Users/benkoch/Desktop/UChicago/2nd year/Spring/ASTR 21200/24inch_data/190526/dark/mdark'

In [4]:
whichpath = datapath

## For ALL the files in the directory.
allfiles = [f for f in os.listdir(datapath)]

allfiles = sorted(allfiles)       ## This is necessary on my Mac, may not be for others?
for i in range(len(allfiles)):
    print( i, allfiles[i])

0 .DS_Store
1 dark_190526_AA-001_020s.fit
2 dark_190526_AA-001_040s.fit
3 dark_190526_AA-001_100s.fit
4 dark_190526_AA-001_180s.fit
5 dark_190526_AA-001_bias.fit
6 dark_190526_AA-002_020s.fit
7 dark_190526_AA-002_040s.fit
8 dark_190526_AA-002_100s.fit
9 dark_190526_AA-002_180s.fit
10 dark_190526_AA-002_bias.fit
11 dark_190526_AA-003_020s.fit
12 dark_190526_AA-003_040s.fit
13 dark_190526_AA-003_100s.fit
14 dark_190526_AA-003_180s.fit
15 dark_190526_AA-003_bias.fit
16 dark_190526_AA-004_020s.fit
17 dark_190526_AA-004_040s.fit
18 dark_190526_AA-004_100s.fit
19 dark_190526_AA-004_180s.fit
20 dark_190526_AA-004_bias.fit
21 dark_190526_AA-005_020s.fit
22 dark_190526_AA-005_040s.fit
23 dark_190526_AA-005_100s.fit
24 dark_190526_AA-005_180s.fit
25 dark_190526_AA-005_bias.fit
26 dark_190526_AA-006_020s.fit
27 dark_190526_AA-006_040s.fit
28 dark_190526_AA-006_100s.fit
29 dark_190526_AA-006_180s.fit
30 dark_190526_AA-006_bias.fit
31 dark_190526_AA-007_020s.fit
32 dark_190526_AA-007_040s.fit
33 da

# This part makes a graph based on ALL the images, each data point is an image. It is also useless. Do not run it.

In [5]:
whichpath = datapath

somefiles = [f for f in os.listdir(whichpath) if '.fit' in f and '_bias' not in f and 'mdark' not in f]

somefiles = sorted(somefiles)       ## This is necessary on my Mac, may not be for others?
for i in range(len(somefiles)):
    print( i, somefiles[i])



0 dark_190526_AA-001_020s.fit
1 dark_190526_AA-001_040s.fit
2 dark_190526_AA-001_100s.fit
3 dark_190526_AA-001_180s.fit
4 dark_190526_AA-002_020s.fit
5 dark_190526_AA-002_040s.fit
6 dark_190526_AA-002_100s.fit
7 dark_190526_AA-002_180s.fit
8 dark_190526_AA-003_020s.fit
9 dark_190526_AA-003_040s.fit
10 dark_190526_AA-003_100s.fit
11 dark_190526_AA-003_180s.fit
12 dark_190526_AA-004_020s.fit
13 dark_190526_AA-004_040s.fit
14 dark_190526_AA-004_100s.fit
15 dark_190526_AA-004_180s.fit
16 dark_190526_AA-005_020s.fit
17 dark_190526_AA-005_040s.fit
18 dark_190526_AA-005_100s.fit
19 dark_190526_AA-005_180s.fit
20 dark_190526_AA-006_020s.fit
21 dark_190526_AA-006_040s.fit
22 dark_190526_AA-006_100s.fit
23 dark_190526_AA-006_180s.fit
24 dark_190526_AA-007_020s.fit
25 dark_190526_AA-007_040s.fit
26 dark_190526_AA-007_100s.fit
27 dark_190526_AA-007_180s.fit
28 dark_190526_AA-008_020s.fit
29 dark_190526_AA-008_040s.fit
30 dark_190526_AA-008_100s.fit
31 dark_190526_AA-008_180s.fit
32 dark_190526_AA-

In [7]:
acceptlist = True             # True if you want to accept all the files in somefiles.
contiguous = True             # True if you want to accept a contiguous subset of somefiles.
startfile, endfile = 5,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

if acceptlist == True:
    files = somefiles
    for i in range(len(files)):
        print(i, files[i])
else:
    if contiguous == True:
        files = somefiles[startfile:endfile+1]
        for i in range(len(files)):
            print( i, files[i])
    else:
        files = []
        for i in range(len(flist)):
            files.append(somefiles[flist[i]])
        for i in range(len(files)):
            print( i, flist[i], files[i])

0 dark_190526_AA-001_020s.fit
1 dark_190526_AA-001_040s.fit
2 dark_190526_AA-001_100s.fit
3 dark_190526_AA-001_180s.fit
4 dark_190526_AA-002_020s.fit
5 dark_190526_AA-002_040s.fit
6 dark_190526_AA-002_100s.fit
7 dark_190526_AA-002_180s.fit
8 dark_190526_AA-003_020s.fit
9 dark_190526_AA-003_040s.fit
10 dark_190526_AA-003_100s.fit
11 dark_190526_AA-003_180s.fit
12 dark_190526_AA-004_020s.fit
13 dark_190526_AA-004_040s.fit
14 dark_190526_AA-004_100s.fit
15 dark_190526_AA-004_180s.fit
16 dark_190526_AA-005_020s.fit
17 dark_190526_AA-005_040s.fit
18 dark_190526_AA-005_100s.fit
19 dark_190526_AA-005_180s.fit
20 dark_190526_AA-006_020s.fit
21 dark_190526_AA-006_040s.fit
22 dark_190526_AA-006_100s.fit
23 dark_190526_AA-006_180s.fit
24 dark_190526_AA-007_020s.fit
25 dark_190526_AA-007_040s.fit
26 dark_190526_AA-007_100s.fit
27 dark_190526_AA-007_180s.fit
28 dark_190526_AA-008_020s.fit
29 dark_190526_AA-008_040s.fit
30 dark_190526_AA-008_100s.fit
31 dark_190526_AA-008_180s.fit
32 dark_190526_AA-

In [ ]:
## Make a stack of images.
image, headlist, rows, cols, imedian, imad = stackfits(datapath,files)

import scipy.stats

exptime = np.zeros(len(headlist))
for i in range(len(headlist)):
    head = headlist[i]
    exptime[i] = head['EXPOSURE']
    
#plt.plot(exptime)

pixel_dark_value = image[:,1000,1000]

#plt.plot(exptime, pixel, 'k.')

corrs = np.zeros((1024,1024))
for i in range(1024):
    for j in range(1024):
        corrs[i,j] = scipy.stats.pearsonr(exptime, image[:,i,j])[0]
    
#print(corrs)

# This part makes a graph just based on the medians. It is the useful part of this code.

In [44]:
whichpath2 = median_path

## For ALL the files in the directory.
allfiles = [f for f in os.listdir(median_path)]

allfiles = sorted(allfiles)       ## This is necessary on my Mac, may not be for others?
for i in range(len(allfiles)):
    print( i, allfiles[i])

0 mdark_190526_AA_020s.fits
1 mdark_190526_AA_040s.fits
2 mdark_190526_AA_100s.fits
3 mdark_190526_AA_180s.fits


In [45]:
whichpath2 = median_path

somefiles = [f for f in os.listdir(whichpath2) if '.fit' in f]

somefiles = sorted(somefiles)       ## This is necessary on my Mac, may not be for others?
for i in range(len(somefiles)):
    print( i, somefiles[i])

0 mdark_190526_AA_020s.fits
1 mdark_190526_AA_040s.fits
2 mdark_190526_AA_100s.fits
3 mdark_190526_AA_180s.fits


In [46]:
acceptlist = True             # True if you want to accept all the files in somefiles.
contiguous = True             # True if you want to accept a contiguous subset of somefiles.
startfile, endfile = 5,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

if acceptlist == True:
    files = somefiles
    for i in range(len(files)):
        print(i, files[i])
else:
    if contiguous == True:
        files = somefiles[startfile:endfile+1]
        for i in range(len(files)):
            print( i, files[i])
    else:
        files = []
        for i in range(len(flist)):
            files.append(somefiles[flist[i]])
        for i in range(len(files)):
            print( i, flist[i], files[i])

0 mdark_190526_AA_020s.fits
1 mdark_190526_AA_040s.fits
2 mdark_190526_AA_100s.fits
3 mdark_190526_AA_180s.fits


In [48]:
## Make a stack of images.
image, headlist, rows, cols, imedian, imad = stackfits(median_path,files)

GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration


rows = 1024    cols = 1024
image.shape = (4, 1024, 1024)


GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration



0 mdark_190526_AA_020s.fits

1 mdark_190526_AA_040s.fits


GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration



2 mdark_190526_AA_100s.fits

3 mdark_190526_AA_180s.fits


In [50]:
import scipy.stats

exptime = np.zeros(len(headlist))
for i in range(len(headlist)):
    head = headlist[i]
    exptime[i] = head['EXPOSURE']
    
#plt.plot(exptime)

pixel_dark_value = image[:,1000,1000]

#plt.plot(exptime, pixel, 'k.')

corrs = np.zeros((1024,1024))
for i in range(1024):
    for j in range(1024):
        corrs[i,j] = scipy.stats.pearsonr(exptime, image[:,i,j])[0]
    
print(corrs)

[[0.9848809  0.99768083 0.99482044 ... 0.93680504 0.97546862 0.99341737]
 [0.99617783 0.99605058 0.96986863 ... 0.84701902 0.96787096 0.99617783]
 [0.99859914 0.99615062 0.96950155 ... 0.98476353 0.98974691 0.96621759]
 ...
 [0.97385168 0.95456558 0.99654333 ... 0.99479712 0.99341737 0.96621759]
 [0.95602222 0.9999523  0.9791224  ... 0.85867758 0.99111258 0.97862901]
 [0.95158927 0.95146299 0.99293693 ... 0.98373875 0.96520097 0.98373875]]


In [52]:
print(scipy.stats.pearsonr(exptime, image[:,0,0]))

(0.9848809044868438, 0.01511909551315616)
